In [ ]:
# dependencies import
from common_dependencies import *
from train_dependencies import *

In [ ]:
# paths for files with original data
PATH_TO_DATA = {
    'run_1': 
        ('data/original_data/run_1/run1_WM32_data.csv',
        'data/original_data/run_1/run1_WM32_defects.csv',
        'data/original_data/run_1/run1_WM32_pipe.csv'),
    'run_2':
        ('data/original_data/run_2/run2_WM32_data.csv',
        'data/original_data/run_2/run2_WM32_defects.csv',
        'data/original_data/run_2/run2_WM32_pipe.csv')
}

CROP_SIZE = 16
CROP_STEP = 2
PART_OF_DATA = 200

# детерминация случайных величин, отвечающих за выбор первоначальных весов и биасов
tf.compat.v1.set_random_seed(290)
tf.random.set_seed(290)
np.random.seed(290)

In [ ]:
data = {}
x_df, y_df = dw.get_x_and_y_data(*PATH_TO_DATA['run_1'])
data['run_1'] = {'x_df': x_df.iloc[:50,:], 'y_df': y_df.iloc[:50,:]}

#x_df, y_df = dw.get_x_and_y_data(*PATH_TO_DATA['run_2'])
#data['run_2'] = {'x_df': x_df, 'y_df': y_df}

In [ ]:
for run, value in data.items():
    dw.draw_defects_map(value['y_df'], title=f'Считанная развернутая карта дефектов для {run}')
    #dw.draw_zeros_quantity_in_data_df(value['x_df'], 
    #                                  title=f'Кол-во 0 значений в считанном датафрейме из данных детекторов для {run}')

In [ ]:
# Take the first left half of each file as train and val dataset
#for run, value in data.items():
#    print('/'*8, run,'/'*8)
#    for data_part_name, data_part in value.items():
#        print(f'{data_part_name} original shape: {value[data_part_name].shape}', end=' -> ')
#        value[data_part_name] = value[data_part_name].iloc[:,:PART_OF_DATA]
#        print(f'sliced shape: {value[data_part_name].shape}')
#    print()

#print(f"run_1.x_df original shape: {data['run_1']['x_df'].shape}")
#print(f"run_1.y_df original shape: {data['run_1']['y_df'].shape}")
#data['run_1']['x_df'] = data['run_1']['x_df'].iloc[:,:PART_OF_DATA]
#data['run_1']['y_df'] = data['run_1']['y_df'].iloc[:,:PART_OF_DATA]
#print(f"run_1.x_df sliced shape: {data['run_1']['x_df'].shape}")
#print(f"run_1.y_df sliced shape: {data['run_1']['y_df'].shape}")

#print()

#print(f"run_2.x_df original shape: {data['run_2']['x_df'].shape}")
#print(f"run_2.y_df original shape: {data['run_2']['y_df'].shape}")
#data['run_2']['x_df'] = data['run_2']['x_df'].iloc[:,-PART_OF_DATA:]
#data['run_2']['y_df'] = data['run_2']['y_df'].iloc[:,-PART_OF_DATA:]
#print(f"run_2.x_df sliced shape: {data['run_2']['x_df'].shape}")
#print(f"run_2.y_df sliced shape: {data['run_2']['y_df'].shape}")

In [ ]:
# extend dfs for better prediction and for crops dividing
for run, value in data.items():
    print('/'*8, run,'/'*8)
    for data_part_name, data_part in value.items():
        print(data_part_name)
        #value[data_part_name] = extend_df_for_prediction(value[data_part_name], 
        #                                                        crop_size=CROP_SIZE, crop_step=CROP_STEP)
        value[data_part_name] = dw.extend_df_for_crops_dividing(value[data_part_name], 
                                                                crop_size=CROP_SIZE, 
                                                                crop_step=CROP_STEP)
    print()

In [ ]:
for run, value in data.items():
    dw.draw_defects_map(value['y_df'], title=f'Расширенная развернутая карта дефектов для {run}')

In [ ]:
# преобразовать данные к виду (измерение, размер х, размер у, каналы)
# для x_data_time каналы = 32
# для x_data_amp каналы = 32
# для y_data каналы = 1
for run, value in data.items():
    print('/'*8, run,'/'*8)
    (x_data_time,
    x_data_amp) = dw.reshape_x_df_to_image_like_numpy(value['x_df'], CROP_SIZE, CROP_STEP)
    value['time_arr'] = x_data_time
    value['amp_arr'] = x_data_amp
    y_data = dw.reshape_y_df_to_image_like_numpy(value['y_df'], CROP_SIZE, CROP_STEP)
    value['mask_arr'] = y_data
    del value['x_df']
    del value['y_df']

In [ ]:
# нормализовать входные данные
for run, value in data.items():
    print('/'*8, run,'/'*8)
    value['time_arr'] = dw.standartize_data(value['time_arr'])
    value['amp_arr'] = dw.standartize_data(value['amp_arr'])

In [ ]:
# Поделить кропы на дефектные и нет
for run, value in data.items():
    print('/'*8, run,'/'*8)
    (x_time_def, x_time_non_def), \
    (x_amp_def, x_amp_non_def), \
    (y_mask_def, y_mask_non_def) = dw.split_def_and_non_def_data(value['time_arr'], 
                                                                 value['amp_arr'], 
                                                                 value['mask_arr'], CROP_SIZE)
    value['def_time_arr'] = x_time_def
    value['non_def_time_arr'] = x_time_non_def
    
    value['def_amp_arr'] = x_amp_def
    value['non_def_amp_arr'] = x_amp_non_def
    
    value['def_mask_arr'] = y_mask_def
    value['non_def_mask_arr'] = y_mask_non_def
    
    del value['time_arr']
    del value['amp_arr']
    del value['mask_arr']

In [ ]:
# take only certain part of non_defect data
'''for run, value in data.items():
    print('/'*8, run,'/'*8)
    shuffled_indices = np.random.permutation(value['non_def_time_arr'].shape[0])
    shuffled_indices = shuffled_indices[:value['def_time_arr'].shape[0]*2]
    
    value['non_def_time_arr'] = value['non_def_time_arr'][shuffled_indices]
    value['non_def_amp_arr'] = value['non_def_amp_arr'][shuffled_indices]
    value['non_def_mask_arr'] = value['non_def_mask_arr'][shuffled_indices]

    for data_part_name, data_part in value.items():
        print(f'{data_part_name}: {data_part.shape}')
    print()'''

In [ ]:
# Аугментировать данные
for run, value in data.items():
    print('/'*8, run,'/'*8)
    for data_part_name, data_part in value.items():
        print(data_part_name)
        value[data_part_name] = dw.augment_data(value[data_part_name])  

In [ ]:
# создание выходных данных бинарного вида (одномерный массив типа bool)
for run, value in data.items():
    print('/'*8, run,'/'*8)
    value['def_binary_arr'] = dw.create_binary_arr_from_mask_arr(value['def_mask_arr'])
    value['non_def_binary_arr'] = dw.create_binary_arr_from_mask_arr(value['non_def_mask_arr'])
    del value['def_mask_arr']
    del value['non_def_mask_arr']

In [ ]:
# деление на тренировочную и валидационную выборки

val_data_percent = 0.2

X_time_train, X_time_val = dw.split_data_to_train_val_datasets([data['run_1']['def_time_arr'],
                                                                data['run_1']['non_def_time_arr']],
                                                                val_data_percent)
del data['run_1']['def_time_arr']
del data['run_1']['non_def_time_arr']
#del data['run_2']['def_time_arr']
#del data['run_2']['non_def_time_arr']

X_amp_train, X_amp_val = dw.split_data_to_train_val_datasets([data['run_1']['def_amp_arr'],
                                                              data['run_1']['non_def_amp_arr']],
                                                              val_data_percent)
del data['run_1']['def_amp_arr']
del data['run_1']['non_def_amp_arr']
#del data['run_2']['def_amp_arr']
#del data['run_2']['non_def_amp_arr']

Y_binary_train, Y_binary_val = dw.split_data_to_train_val_datasets([data['run_1']['def_binary_arr'],
                                                                   data['run_1']['non_def_binary_arr']],
                                                                   val_data_percent)
del data['run_1']['def_binary_arr']
del data['run_1']['non_def_binary_arr']
#del data['run_2']['def_binary_arr']
#del data['run_2']['non_def_binary_arr']
data = None

In [ ]:
# перемешивание данных
train_shuffled_indices = np.random.permutation(X_time_train.shape[0])
val_shuffled_indices = np.random.permutation(X_time_val.shape[0])

X_time_train = X_time_train[train_shuffled_indices]
X_amp_train = X_amp_train[train_shuffled_indices]
Y_binary_train = Y_binary_train[train_shuffled_indices]

X_time_val = X_time_val[val_shuffled_indices]
X_amp_val = X_amp_val[val_shuffled_indices]
Y_binary_val = Y_binary_val[val_shuffled_indices]

In [ ]:
print(f'{X_time_train.shape=}')
print(f'{X_amp_train.shape=}')
print(f'{Y_binary_train.shape=}\n')

print(f'{X_time_val.shape=}')
print(f'{X_amp_val.shape=}')
print(f'{Y_binary_val.shape=}')

In [ ]:
# всякие константы для последующей работы

#///////////////////////////////// для компиляции 

CMP_learning_rate = 0.000004 #0.0000002 # шаг сходимости back propogation
CMP_solver = keras.optimizers.Adam(CMP_learning_rate) # оптимизатор
#CMP_solver = keras.optimizers.SGD(CMP_learning_rate) # оптимизатор
CMP_loss_func = 'mean_squared_error'# функция потерь
#///////////////////////////////// для колбэков

    # для Early_stopping
ES_patience = 3 # кол-во эпох без улучшений
ES_min_delta = 0.00005 # минимальное улучшение параметра за cur_patience
ES_monitor_parametr =  'loss' # отслеживаемый параметр 
ES_save_best_weights = True # сохранять ли веса нейронки с лучшими результатами
    
    # для ReduceLROnPlateau
RLPOP_monitor_parametr = 'loss'  # отслеживаемый параметр 
RLPOP_factor = 0.2 # множитель для расчета нового шага сходимости (new_learning_rate = old_learning_rate*RLPOP_factor)
RLPOP_patience = 1 # кол-во эпох без улучшений
RLPOP_verbose = 1 # выводить ли прогресс изменения шага сходимости в его процессее
RLPOP_mode = 'auto' # выбирает, уменьшать шаг сходимости при росте величины или при её уменьшении
RLPOP_min_delta = 0.001 # порог изменения отслеживаемого значения
RLPOP_cooldown = 2 # количество эпох до возобновления работы после изменения шага сходимости
RLPOP_min_lr = 0 # минимальное значение шага сходимости

    # для CallbackList
CBL_add_history = True # вызывать ли колбэк History (если он не был довавлен вручную)
CBL_add_progbar = True # вызывать ли колбэк ProgbarLogger (если он не был довавлен вручную)
    
#///////////////////////////////// для тренировки

FIT_batch_size = 48 # размер bach при обучении/тестировании1
#FIT_shuffle = True # перемешивать ли данные
FIT_verbose = True # выводить ли прогресс обучения в его процессее
FIT_epochs = 30 # количество эпох обучения
#FIT_validation_split = 0.10 #0.20 # процент валидационных данных, отсекаемых из тестовой выборки

In [ ]:
# создание архитектуры модели
# 1 подсеть //////////////////////////////////////////////////
input_time = Input((16,16,32), name = 'input_time')

# 12
dconv_1_1 = Conv2D(64, (3,3), dilation_rate=(2, 2), activation='relu', name='dconv_1_1')(input_time)

# 6
dconv_1_2 = Conv2D(64, (3,3), dilation_rate=(5, 5), activation='relu', name='dconv_1_2')(input_time)
up_1_2 = UpSampling2D(2, interpolation='bilinear', name='up_1_2') (dconv_1_2)

# 4
dconv_1_3 = Conv2D(64, (3,3), dilation_rate=(6, 6), activation='relu', name='dconv_1_3')(input_time)
up_1_3 = UpSampling2D(3, interpolation='bilinear', name='up_1_3') (dconv_1_3)

# 2
dconv_1_4 = Conv2D(64, (3,3), dilation_rate=(7, 7), activation='relu', name='dconv_1_4')(input_time)
up_1_4 = UpSampling2D(6, interpolation='bilinear', name='up_1_4') (dconv_1_4)

conc_1_1 = concatenate([dconv_1_1, up_1_2, up_1_3, up_1_4],axis=3, name='conc_1_1')

conv_1_1 = Conv2D(512, (3,3), activation='relu', name='conv_1_1')(conc_1_1)
conv_1_2 = Conv2D(512, (3,3), activation='relu', name='conv_1_2')(conv_1_1)
pool_1_1 = MaxPooling2D((2,2), strides=2, name='pool_1_1')(conv_1_2)

conv_1_3 = Conv2D(1024, (3,3), activation='relu', name='conv_1_3')(pool_1_1)
pool_1_2 = MaxPooling2D((2,2), strides=2, name='pool_1_2')(conv_1_3)

# 2 подсеть //////////////////////////////////////////////////
input_amp = Input((16,16,32), name = 'input_amp')

# 12
dconv_2_1 = Conv2D(64, (3,3), dilation_rate=(2, 2), activation='linear', name='dconv_2_1')(input_amp)

# 6
dconv_2_2 = Conv2D(64, (3,3), dilation_rate=(5, 5), activation='linear', name='dconv_2_2')(input_amp)
up_2_2 = UpSampling2D(2, interpolation='bilinear', name='up_2_2') (dconv_2_2)

# 4
dconv_2_3 = Conv2D(64, (3,3), dilation_rate=(6, 6), activation='linear', name='dconv_2_3')(input_amp)
up_2_3= UpSampling2D(3, interpolation='bilinear', name='up_2_3') (dconv_2_3)

# 2
dconv_2_4 = Conv2D(64, (3,3), dilation_rate=(7, 7), activation='linear', name='dconv_2_4')(input_amp)
up_2_4 = UpSampling2D(6, interpolation='bilinear', name='up_2_4') (dconv_2_4)

conc_2_1 = concatenate([dconv_2_1, up_2_2, up_2_3, up_2_4],axis=3, name='conc_2_1')

conv_2_1 = Conv2D(512, (3,3), activation='linear', name='conv_2_1')(conc_2_1)
conv_2_2 = Conv2D(512, (3,3), activation='linear', name='conv_2_2')(conv_2_1)
pool_2_1 = MaxPooling2D((2,2), strides=2, name='pool_2_1')(conv_2_2)

conv_2_3 = Conv2D(1024, (3,3), activation='linear', name='conv_2_3')(pool_2_1)
pool_2_2 = MaxPooling2D((2,2), strides=2, name='pool_2_2')(conv_2_3)

# выходная подсеть //////////////////////////////////////////////////

conc_3_1 = concatenate([pool_1_2, pool_2_2], axis=3, name='conc_3_1')
flat_3_1 = Flatten(name='flat')(conc_3_1)

d_3_1 = Dense(4096, activation='linear', name='d_3_1')(flat_3_1)
d_3_2 = Dense(1024, activation='linear', name='d_3_2')(d_3_1)
d_3_3 = Dense(128, activation='linear', name='d_3_3')(d_3_2)
d_3_4 = Dense(32, activation='linear', name='d_3_4')(d_3_3)

output_3_1 = Dense(1, activation='sigmoid', name='output_3_1')(d_3_4)

model = keras.Model([input_time, input_amp], output_3_1, name='model')

#model = keras.models.load_model(f"networks/CNN/id=v04n03_in(16x16+16x16)_out(1)_train=0dot0_test=0dot0.h5")

model.compile(optimizer=CMP_solver, loss=CMP_loss_func)

print(model.summary())

In [ ]:
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
plot_model(
    model,
    to_file="model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=200,
    show_layer_activations=False,
    show_trainable=False,
)

In [ ]:
# Создание и настройка колбэков

callback_list = [] # массив колбэков до подачи в колбек "callbacklist"

temp = keras.callbacks.EarlyStopping(
            monitor = ES_monitor_parametr, 
            min_delta = ES_min_delta, 
            patience = ES_patience,
            restore_best_weights = ES_save_best_weights
            )
callback_list.append(temp)

temp = keras.callbacks.ReduceLROnPlateau(
            monitor = RLPOP_monitor_parametr, 
            factor = RLPOP_factor, 
            patience = RLPOP_patience, 
            verbose = RLPOP_verbose,
            mode = RLPOP_mode, 
            min_delta = RLPOP_min_delta, 
            cooldown = RLPOP_cooldown, 
            min_lr = RLPOP_min_lr
            )
callback_list.append(temp)

FIT_callback_list = keras.callbacks.CallbackList(
            callbacks = callback_list, 
            add_history = CBL_add_history, 
            add_progbar = CBL_add_progbar, 
            model = model
            )

In [ ]:
history = model.fit([X_time_train, X_amp_train], Y_binary_train,
                    batch_size = FIT_batch_size, 
                    epochs = FIT_epochs, 
                    verbose = FIT_verbose, 
                    validation_data = ([X_time_val, X_amp_val], Y_binary_val), 
                    callbacks = FIT_callback_list)

In [ ]:
fig, ax = plt.subplots()

fig.set_figwidth(12)
fig.set_figheight(8)

plt.plot(history.history['loss'], 
         label='Тренировочная выборка',  linewidth=1.5, color='blue')
plt.plot(history.history['val_loss'], linestyle = '--', 
         label='Валидационная выборка',  linewidth=3, color='red')

ax.set_xlabel('Номер эпохи', fontsize=20)
ax.set_ylabel('Значение функции ошибки', fontsize=20)
ax.set_title(f"График значений функции ошибки во время обучения", fontsize=20, pad=15)

ax.patch.set_alpha(0)

#  Устанавливаем форматирование делений:
ax.tick_params(axis='both', which='both', labelsize = 20)

# Вывод и настройка сетки
ax.minorticks_on()
ax.grid(which='major', linewidth=2)
ax.grid(which='minor', color = 'gray', linestyle = ':')

ax.legend(fontsize = 20, facecolor = "white", loc = 'upper right')

plt.show()

In [ ]:
# сохранение модели в файл
model.save(f"networks/CNN/id=v04n06_in(16x16+16x16)_out(1)_train=0dot0_test=0dot0.h5", save_format="h5")

#### 